In [191]:
from scipy.spatial import Delaunay
import numpy as np
import scipy as sp
from matplotlib import pyplot as plt
from matplotlib import cm
import time
%matplotlib

Using matplotlib backend: Qt5Agg


In [192]:
"""
create class called grid to collect the grids
create flattened grid of structure (ngrid**3, 3 - xyz)
"""

class grid:
    """
    Generates a grid object with both flattened and mesh versions of cartesian
    """
    def __init__(self, ngrid_cart, ngrid_tor, R0):
        self.R, self.theta, self.phi = np.meshgrid(np.linspace(1e-2, 1,ngrid_tor), 
                                                   np.linspace(0,2*np.pi, ngrid_tor), 
                                                   np.linspace(0,2*np.pi, 2*ngrid_tor),
                                                   indexing='ij')
        self.cart_x, self.cart_y, self.cart_z = np.meshgrid(np.linspace(-3,3, ngrid_cart),
                                                            np.linspace(-3,3, ngrid_cart),
                                                            np.linspace(-1,1, ngrid_cart),
                                                            indexing='ij')
        self.tor_x = (R0 + self.R*np.cos(self.theta))*np.cos(self.phi)
        self.tor_y = (R0 + self.R*np.cos(self.theta))*np.sin(self.phi)
        self.tor_z = self.R*np.sin(self.theta)
        
        self.cartflat = np.zeros((ngrid_cart**3, 3))
        self.cartflat[:,0] = self.cart_x.flatten()
        self.cartflat[:,1] = self.cart_y.flatten()
        self.cartflat[:,2] = self.cart_z.flatten()
        
        self.torflat = np.zeros((2*ngrid_tor**3, 3))
        self.torflat[:,0] = self.tor_x.flatten()
        self.torflat[:,1] = self.tor_y.flatten()
        self.torflat[:,2] = self.tor_z.flatten()
    def plotgrid(self):
        fig_gridscat = plt.figure()
        ax_gridscat = fig_gridscat.add_subplot(projection='3d')
        ax_gridscat.scatter(self.cart_x, self.cart_y, self.cart_z, marker='.')
        ax_gridscat.scatter(self.tor_x, self.tor_y, self.tor_z, marker='x')
        ax_gridscat.set_xlabel("$x[\\mathrm{m}]$")
        ax_gridscat.set_ylabel("$y[\\mathrm{m}]$")
        ax_gridscat.set_zlabel("$z[\\mathrm{m}]$")
    def delaunay_tessalation(self):
        tri = Delaunay(self.torflat)
        idx_simplices = tri.simplices
        coord_simplices = self.torflat[idx_simplices]
        return tri, idx_simplices, coord_simplices
        
# grid1 = grid(ngrid_cart = 30, ngrid_tor = 10, R0 = 2)
grid1 = grid(30, 10, 2)

print(grid1.torflat)

[[ 2.01000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.90109266e+00  6.52645933e-01  0.00000000e+00]
 [ 1.58617242e+00  1.23456755e+00  0.00000000e+00]
 ...
 [ 2.36742153e+00 -1.84263814e+00 -2.44929360e-16]
 [ 2.83745173e+00 -9.74098408e-01 -2.44929360e-16]
 [ 3.00000000e+00 -7.34788079e-16 -2.44929360e-16]]


In [193]:
"""
To check that the object function works
"""
# grid1.plotgrid()

'\nTo check that the object function works\n'

In [194]:
"""
Delaunay triangulation.
"""
tri1, idx_tri1, coord_tri1 = grid1.delaunay_tessalation()

#extract the coordinates of the simplices
print(coord_tri1.shape)

(11887, 4, 3)


In [6]:
# fig_simplices = plt.figure()
# ax_simplices = fig_simplices.add_subplot(projection = '3d')
# ax_simplices.scatter(coord_tri1[:,:,0], coord_tri1[:,:,1], coord_tri1[:,:,2])

In [195]:
"""
To check if the tri object function find_simplex that finds the tetrahedron which a cartesian point is in
we plot the point (2,0,0) to see of the correct tetrahedron is found.
"""
eucdist_200 = np.sqrt(np.sum((grid1.cartflat - np.array([2,0,0], dtype = np.float32))**2, axis = 1))
print(eucdist_200.shape)
idx_200 = np.where(np.isclose(eucdist_200, 0, atol = 1.75e-1) == True )[0]
print("shape of indices atol = 1.75e-1 close to point (2,0,0)", idx_200.shape)
print("the distances to the point are", eucdist_200[idx_200])
print("The point chosen:", grid1.cartflat[idx_200[0]])

idx_simplex_200 = tri1.find_simplex(grid1.cartflat[idx_200[0]])
print(idx_simplex_200)
coord_simplex_200 = coord_tri1[idx_simplex_200]
print(coord_simplex_200)

neighbors_200 = tri1.neighbors[idx_simplex_200]
coords_neighbors200 = coord_tri1
print(coords_neighbors200)
print(coords_neighbors200.shape)

(27000,)
shape of indices atol = 1.75e-1 close to point (2,0,0) (8,)
the distances to the point are [0.15030686 0.11436637 0.11436637 0.15030686 0.15030686 0.11436637
 0.11436637 0.15030686]
The point chosen: [ 1.96551724 -0.10344828 -0.10344828]
7356
[[ 1.88690540e+00 -6.47775441e-01 -8.66025404e-03]
 [ 1.94000000e+00 -4.75162958e-16 -1.03923048e-01]
 [ 2.02083778e+00  0.00000000e+00 -1.18176930e-01]
 [ 1.91134322e+00 -6.56164955e-01 -1.18176930e-01]]
[[[ 2.68177955e+00 -6.56846549e-16 -5.72080973e-01]
  [ 2.83745173e+00 -9.74098408e-01 -2.44929360e-16]
  [ 2.76604444e+00 -6.77485495e-16 -6.42787610e-01]
  [ 3.00000000e+00  0.00000000e+00  0.00000000e+00]]

 [[ 2.68177955e+00 -6.56846549e-16 -5.72080973e-01]
  [ 2.89000000e+00  0.00000000e+00  0.00000000e+00]
  [ 2.83745173e+00 -9.74098408e-01 -2.44929360e-16]
  [ 3.00000000e+00  0.00000000e+00  0.00000000e+00]]

 [[ 2.68177955e+00 -6.56846549e-16 -5.72080973e-01]
  [ 2.61617253e+00 -8.98133162e-01 -6.42787610e-01]
  [ 2.83745173e+00 

In [196]:
from scipy.spatial import ConvexHull
import matplotlib as mpl
import mpl_toolkits.mplot3d as a3

"""
Plot the tetrahedron to confirm that we have found which tetrahedron the point is in.
Note!!! If a point is located in one of the faces of a tetrahedron the point will belong a
degenerate number of tetrahedra. The worst case here is if a point is located at one of the vertices.
Then the degeneracy will be all tetrahadra with that point as a vertex.
"""

def plot_tetrhedron_and_interiorpnt(coord_simplex, idx_p):
    hull_found = ConvexHull(coord_simplex)
    indices_found = hull_found.simplices
    faces_found = coord_simplex[indices_found]
    print(faces_found)
    fig_foundsimplex = plt.figure()
    ax_foundsimplex = fig_foundsimplex.add_subplot(projection='3d')
    ax_foundsimplex.set_xlim(-3,3)
    ax_foundsimplex.set_ylim(-3,3)
    ax_foundsimplex.set_zlim(-1,1)
    ax_foundsimplex.set_xlabel("$x[\\mathrm{m}]$")
    ax_foundsimplex.set_ylabel("$y[\\mathrm{m}]$")
    ax_foundsimplex.set_zlabel("$z[\\mathrm{m}]$")

    for f in faces_found:
        face = a3.art3d.Poly3DCollection([f])
        face.set_color(mpl.colors.rgb2hex(np.random.rand(3)))
        face.set_edgecolor('k')
        face.set_alpha(0.5)
        ax_foundsimplex.add_collection3d(face)

    ax_foundsimplex.scatter(grid1.cartflat[idx_p,0], 
                            grid1.cartflat[idx_p,1], 
                            grid1.cartflat[idx_p,2], marker='x')

print(idx_200[0])
# plot_tetrhedron_and_interiorpnt(coord_simplex = coord_simplex_200, 
#                                 idx_p = idx_200[0])

22033


In [197]:
"""
Find all tetrahedra which a cartesian point belongs to. The triangle object function returns -1
if the point does not belong to any tetrahedra.
Creating a dictionary with key coord, value vertices of the tetrahedron in which the point is interior
"""
def make_dict_coord_vertices(grid_obj, tri_obj = None):

    dict_p_tetrahedron = {}
    ncartcoord = int(len(grid_obj.cartflat))
    if tri_obj == None:
        tri, idx_tri, coord_tri = grid_obj.delaunay_tessalation()
    else:
        tri = tri_obj
        idx_tri = tri.simplices
        coord_tri = grid_obj.torflat[idx_tri]
    for i in range(ncartcoord):
        p = grid_obj.cartflat[i]
        idx_simplex_embed_p = tri.find_simplex(p)
        if idx_simplex_embed_p != -1:
            dict_p_tetrahedron[tuple(p)] = coord_tri[idx_simplex_embed_p]
    return dict_p_tetrahedron

dict_p_tetrahedron1 = make_dict_coord_vertices(grid_obj = grid1)

In [198]:
print("The number of coordinates that is interior to a tetrahedron:", len(dict_p_tetrahedron1))

The number of coordinates that is interior to a tetrahedron: 15356


In [199]:
"""
Test: We want to plot a random point and tetrahedron of which the point is embedded by.
"""
def test_plot_rnd_pnt_tetra(grid_obj, dict_p_tetra):
    idx_rand = np.random.randint(low=0, high=grid_obj.cartflat.shape[0]-1, size = 1)
    coord_rand = grid1.cartflat[idx_rand,:][0]
    print(coord_rand)
    try:
        plot_tetrhedron_and_interiorpnt(coord_simplex = dict_p_tetra[tuple(coord_rand)], idx_p = idx_rand)
    except KeyError:
        print("The point is not inside any of the tetrahedra")
    return coord_rand

In [203]:
test_rndpnt = test_plot_rnd_pnt_tetra(grid_obj = grid1, dict_p_tetra = dict_p_tetrahedron1)

[-1.34482759 -0.72413793  0.17241379]
[[[-1.51269485 -0.81862952  0.48497423]
  [-1.29614391 -0.701438    0.19153128]
  [-1.55562821 -0.25958844  0.15390906]]

 [[-1.38705176 -0.75063487  0.15390906]
  [-1.29614391 -0.701438    0.19153128]
  [-1.55562821 -0.25958844  0.15390906]]

 [[-1.38705176 -0.75063487  0.15390906]
  [-1.51269485 -0.81862952  0.48497423]
  [-1.55562821 -0.25958844  0.15390906]]

 [[-1.38705176 -0.75063487  0.15390906]
  [-1.51269485 -0.81862952  0.48497423]
  [-1.29614391 -0.701438    0.19153128]]]


In [204]:
"""
Now that we have found all coordinates and 
the tetrahedra of which they are interior, we want to assign values to each point
interior to each tetrahedra by using an interpolation scheme.

Interpolation scheme 1:

Weighted values by normalized inverse distance to each vertex of the tetrahedron.
By normalized we mean the sum of the inverse distances should equal 1.
"""

def interpolation_p_in_simplex(p, simplex, dict_vals):
    """
    args:
    p - (x,y,z) cart coords of p in tetrahedron/simplex, dim (3,)
    simplex - (x,y,z) cart coord of each vertex of tetrahedron/simplex, dim (4,3)
    dict_vals - dictionary value scalar valuesof potential, key coord of a vertex, 
                len = number of grid points in toroidal coord
    out:
    paramval = scalar, param value interpolated from param vals at vertices, dim (1,)
    """
    vals = []
    for i in range(len(simplex)):
        vals.append(dict_vals[tuple(simplex[i])])
    vals = np.asarray(vals, np.float64)
    euc_dists = np.sqrt(np.sum((simplex - p)**2, axis = 1)) + 1e-8
    N = 1/(np.sum(1/euc_dists, axis=0))
    paramval = np.sum(N*vals/euc_dists, axis = 0)
    return paramval

In [205]:
# run a test if the function works on a random point interior to a tetrahedron
print("Coord: ", test_rndpnt)
test_potvals = {tuple(dict_p_tetrahedron1[tuple(test_rndpnt)][0]):1, 
                tuple(dict_p_tetrahedron1[tuple(test_rndpnt)][1]):2, 
                tuple(dict_p_tetrahedron1[tuple(test_rndpnt)][2]):3, 
                tuple(dict_p_tetrahedron1[tuple(test_rndpnt)][3]):4}
print("Coord vertices of tetrahedron with potvals:\n", test_potvals)
test_interpolationvals = interpolation_p_in_simplex(p = test_rndpnt,
                                                    simplex = dict_p_tetrahedron1[tuple(test_rndpnt)], 
                                                    dict_vals = test_potvals)
print(test_interpolationvals)

Coord:  [-1.34482759 -0.72413793  0.17241379]
Coord vertices of tetrahedron with potvals:
 {(-1.5556282095991059, -0.2595884357028291, 0.153909064496551): 1, (-1.3870517550303403, -0.7506348721704941, 0.153909064496551): 2, (-1.5126948520751613, -0.8186295160237664, 0.4849742261192857): 3, (-1.29614390567014, -0.701438004327379, 0.1915312802623746): 4}
2.8736093346831284


In [206]:
"""
Define some potential that is easy to interpret. Just to have some values to interpolate from.
"""

def testpot(r):
    return np.exp(-r)

tpot_tor = testpot(r = grid1.R)

tpot_cart = testpot(r = np.sqrt((np.sqrt(grid1.cart_x**2 + grid1.cart_y**2) - 2)**2 + grid1.cart_z**2))

In [224]:
"""
Now define a potential which varies in all directions. The potential varies periodically in theta and phi direction.
"""
def tpot_alldir_cart(grid_obj):
    r = np.sqrt((np.sqrt(grid_obj.cart_x**2 + grid_obj.cart_y**2) - 2)**2 + grid_obj.cart_z**2)
    phi = np.arctan2(grid_obj.cart_y,grid_obj.cart_x)
    theta = np.arctan2(grid_obj.cart_z,r)
    return np.exp(-r)*np.sin(theta)*np.cos(phi)

def tpot_alldir_tor(grid_obj):
    return np.exp(-grid_obj.R)*np.sin(grid_obj.theta)*np.cos(grid_obj.phi)

In [226]:
testpot_alldir_cart = tpot_alldir_cart(grid_obj = grid1)
testpot_alldir_tor = tpot_alldir_tor(grid_obj = grid1)

In [210]:
"""
Function to plot a param as a contour plot for inspection.
Param must be on a meshgrid structure.
"""
def plot_paramslice_cart(param, idx_y):
    im = plt.imshow(param[:,idx_y].T)
    plt.colorbar(im)
# plot_paramslice_cart(param = tpot_cart, idx_y = 15)

In [211]:
def plot_paramslice_tor(grid_obj, param, idx_phi):
    fig = plt.figure()
    ax = fig.add_subplot()
    c = ax.contourf(np.sqrt(grid_obj.tor_x[:,:,idx_phi]**2 - grid_obj.tor_y[:,:,idx_phi]**2) - 2, 
                    grid_obj.tor_z[:,:,idx_phi], param[:,:,idx_phi])
    fig.colorbar(c)
# plot_paramslice_tor(grid_obj = grid1, param = tpot_tor, idx_phi = 0)

In [217]:
# plot_paramslice_cart(param = testpot_alldir_cart, idx_y = 10)

In [218]:
# plot_paramslice_tor(grid_obj = grid1, param = testpot_alldir_tor, idx_phi = 0)

In [17]:
"""
Now test the interpolation method wiht actual potential data from the exponentially decaying potential.
"""
def make_param_mesh(dict_p_tetra, grid_obj, tpot_torgrid):
    """
    Make a dictionary with key = coord xyz and value = potential value 
    of the potential values connected to grid points from the toroidal grid.

    """
    dict_coord_tpot = {}
    for i in range(len(grid_obj.torflat)):
        dict_coord_tpot[tuple(grid_obj.torflat[i])] = tpot_torgrid.flatten()[i]

    # Points interior to one of the tetrahedra that the toroidal volume is divided in
    ps_interior = np.asarray(list(dict_p_tetra.keys()), dtype = np.float64)

    # Make a dictionary with key coord, value interpolated parameter value
    dict_coord_pot_interpol = {}
    
    # Make a meshgrid of interpolated parameter values
    potmesh = np.nan*np.ones(grid_obj.cart_x.shape)
    
    for i in range(len(ps_interior)):
        coord_idx = np.argwhere((grid_obj.cart_x == ps_interior[i][0]) & 
                                (grid_obj.cart_y == ps_interior[i][1]) & 
                                (grid_obj.cart_z == ps_interior[i][2]))[0] 
        potmesh[coord_idx[0], coord_idx[1], coord_idx[2]] = interpolation_p_in_simplex(p = ps_interior[i],
                                                            simplex = dict_p_tetra[tuple(ps_interior[i])], 
                                                            dict_vals = dict_coord_tpot)
        dict_coord_pot_interpol[tuple(ps_interior[i])] = potmesh[coord_idx[0], coord_idx[1], coord_idx[2]]

    return potmesh, dict_coord_pot_interpol

In [18]:
potmesh1, dict_coord_pot_interpol1 = make_param_mesh(dict_p_tetra = dict_p_tetrahedron1, 
                                                     grid_obj = grid1, 
                                                     tpot_torgrid = tpot_tor)

In [19]:
potmesh1.shape
# np.max(np.abs(potmesh1-tpot_cart))

(30, 30, 30)

In [20]:
"""
Test the interpolation on one point to and compare with the direct solution on the cartesian grid.
"""
idx_draw = [27,15,15]

print("From direct solution of potential on cartesian grid the coords chosen with potential value are: \n")
print("x:", grid1.cart_x[idx_draw[0], idx_draw[1], idx_draw[2]])
print("y: ", grid1.cart_y[idx_draw[0], idx_draw[1], idx_draw[2]])
print("z: ", grid1.cart_z[idx_draw[0], idx_draw[1], idx_draw[2]])
print("p:", tpot_cart[idx_draw[0], idx_draw[1], idx_draw[2]])

print("\nFrom interpolated solution of potential on cartesian grid the coords chosen with potential value are: \n")
print("xyz", grid1.cartflat[idx_draw[2] + idx_draw[1]*grid1.cart_x.shape[1] + idx_draw[0]*grid1.cart_x.shape[0]**2])
print("p", dict_coord_pot_interpol1[tuple(grid1.cartflat[idx_draw[2] + idx_draw[1]*grid1.cart_x.shape[1] + idx_draw[0]*grid1.cart_x.shape[0]**2])])

From direct solution of potential on cartesian grid the coords chosen with potential value are: 

x: 2.586206896551724
y:  0.10344827586206895
z:  0.034482758620689724
p: 0.5547238804238811

From interpolated solution of potential on cartesian grid the coords chosen with potential value are: 

xyz [2.5862069  0.10344828 0.03448276]
p 0.5383733574324316


In [168]:
"""
Restructure interpolated potential data in mesh format to plot the potential in a contourplot to
be able to visually compare the direct solution on the cartesian grid to the interpolated values.

Current issue:
I have to restructure the interpolated parameter(potential) values in a mesh such that it is plotable.
"""


fig_testpot_interpol = plt.figure()
ax_testpot_interpol = fig_testpot_interpol.add_subplot(projection='3d')
potscat = ax_testpot_interpol.scatter(grid1.cartflat[:,0], 
                                      grid1.cartflat[:,1], 
                                      grid1.cartflat[:,2], 
                                      c = potmesh1 - tpot_cart, cmap = cm.jet)

ax_testpot_interpol.set_xlabel("$x[m]$")
ax_testpot_interpol.set_ylabel("$y[m]$")
ax_testpot_interpol.set_zlabel("$z[m]$")
fig_testpot_interpol.colorbar(potscat)

In [321]:
"""
Use MMS to confirm that error of grid converges as you increase the number of gridpoints.
Strategy:
1. Establish all grids with increasing number of points in the cartesian grid.
2. Calculate the interpolation
3. Compare potential values with the direct solution on the cartesian grid and compute a 1D measure of the error.
4. Plot error in loglog and see if the error converges.

"""
tik = time.time()

gridlist = []
tpot_cartlist = []
tpot_torlist = []
for i in range(4):
    gridlist.append( grid(ngrid_cart = 20, ngrid_tor = 10*(i+1), R0 = 2) )
    tpot_cartlist.append( testpot(r = np.sqrt((np.sqrt(gridlist[i].cart_x**2 + gridlist[i].cart_y**2) - 2)**2 + 
                                              gridlist[i].cart_z**2)) )
    tpot_torlist.append( testpot(r = gridlist[i].R) )

tok = time.time()
print("This cell use ", tok - tik, "seconds to run.")

This cell use  0.028984785079956055 seconds to run.


In [167]:
"""
Test the delaunay tessalation on a generalized 3d romboid.
Here you should se that a generalized 3d romboid gets split into 6 tetrahedra.
"""
xx_rom3d, yy_rom3d, zz_rom3d = np.meshgrid(np.array([0,1,0,1,0,1,0,1]), 
                                            np.array([0,0,1,1,0,0,1,1]), 
                                            np.array([0,0,0,0,1,1,1,1]))
rom3d_gridflat = np.zeros((grid1.cart_x[:2, :2, :2].shape[0]**3,) + (3,))
rom3d_gridflat[:,0] = grid1.tor_x[4:(4+2), :2, :2].flatten()
rom3d_gridflat[:,1] = grid1.tor_y[4:(4+2), :2, :2].flatten()
rom3d_gridflat[:,2] = grid1.tor_z[4:(4+2), :2, :2].flatten()


tri_rom3d = Delaunay(rom3d_gridflat)
# print("The shape of the resulting tessalation, should be (6,4)", tri_rom3d.simplices.shape)
coords_simplices = rom3d_gridflat[tri_rom3d.simplices]
coords_simplices.shape

# # plot the vertices of the cell where we check for an interior point.
# figrom3d = plt.figure()
# axrom3d = figrom3d.add_subplot(projection='3d')
# axrom3d.scatter(coords_simplices[:,:,0], coords_simplices[:,:,1], coords_simplices[:,:,2])


def plot_tetrhedron_and_interiorpnt(coord_simplices):
    fig_foundsimplex = plt.figure()
    ax_foundsimplex = fig_foundsimplex.add_subplot(projection='3d')
    ax_foundsimplex.set_xlim(-3,3)
    ax_foundsimplex.set_ylim(-3,3)
    ax_foundsimplex.set_zlim(-1,1)
    ax_foundsimplex.set_xlabel("$x[\\mathrm{m}]$")
    ax_foundsimplex.set_ylabel("$y[\\mathrm{m}]$")
    ax_foundsimplex.set_zlabel("$z[\\mathrm{m}]$")
    for coord_simplex in coord_simplices:
        hull_found = ConvexHull(coord_simplex)
        indices_found = hull_found.simplices
        faces_found = coord_simplex[indices_found]

        for f in faces_found:
            face = a3.art3d.Poly3DCollection([f])
            face.set_color(mpl.colors.rgb2hex(np.random.rand(3)))
            face.set_edgecolor('k')
            face.set_alpha(0.5)
            ax_foundsimplex.add_collection3d(face)
# plot_tetrhedron_and_interiorpnt(coord_simplices = coords_simplices)

In [36]:
"""
Check visually that only one box is isolated each iteration of the manual tessalation.
Check one iteration of the manual tessalation and confirm that inteior points are acutally
interoir to the toroidal grid points.
"""

# n=3
# ir = 0
# it = 10
# ip = 0
# gridflat = np.zeros((n**3,) + (3,))
# gridflat[:,0] = grid2.tor_x[ir:(ir + n), it:(it + n), ip:(ip + n)].flatten()
# gridflat[:,1] = grid2.tor_y[ir:(ir + n), it:(it + n), ip:(ip + n)].flatten()
# gridflat[:,2] = grid2.tor_z[ir:(ir + n), it:(it + n), ip:(ip + n)].flatten()
# print("shape gridflat", gridflat.shape)
# tri1 = Delaunay(gridflat)
# idx_tri1 = tri1.simplices
# coord_tri1 = gridflat[idx_tri1]
# print(coord_tri1)
# print(gridflat)

# minx = np.min(gridflat[:,0])
# maxx = np.max(gridflat[:,0])
# miny = np.min(gridflat[:,1])
# maxy = np.max(gridflat[:,1])
# minz = np.min(gridflat[:,2])
# maxz = np.max(gridflat[:,2])

# print("minx", minx, "maxx", maxx, "miny", miny, "maxy", maxy, "minz", minz, "maxz", maxz)
# epsx = grid2.cart_x[1,0,0] - grid2.cart_x[0,0,0]
# epsy = grid2.cart_y[0,1,0] - grid2.cart_y[0,0,0]
# epsz = grid2.cart_z[0,0,1] - grid2.cart_z[0,0,0]

# idx_inside = np.argwhere((minx - epsx < grid2.cartflat[:,0]) & (maxx + epsx > grid2.cartflat[:,0]) & 
#                          (miny - epsy < grid2.cartflat[:,1]) & (maxy + epsy > grid2.cartflat[:,1]) & 
#                          (minz - epsz < grid2.cartflat[:,2]) & (maxz + epsz > grid2.cartflat[:,2]) )
# print("idx_inside", idx_inside.shape)
# dict_p_tetrahedron_test1 = {}

# for i in idx_inside:
#     p = grid2.cartflat[i][0]
#     idx_simplex_embed_p1 = tri1.find_simplex(p)
#     if idx_simplex_embed_p1 != -1:
#         p_tuple = tuple(list(p))
#         dict_p_tetrahedron_test1[p_tuple] = coord_tri1[idx_simplex_embed_p1] 


# print(dict_p_tetrahedron_test1)

'\nCheck visually that only one box is isolated each iteration of the manual tessalation.\nCheck one iteration of the manual tessalation.\n'

In [37]:
"""
Scatter plot of the interior points with the vertices of the simplices.
"""

# fig1 = plt.figure()
# ax1 = fig1.add_subplot(projection='3d')
# ax1.scatter(gridflat[:,0], gridflat[:,1], gridflat[:,2], color = 'k')
# for point in np.array(list(dict_p_tetrahedron_test1.keys())):
#     ax1.scatter(point[0], point[1], point[2], color = 'y')

In [38]:
def manual_tessalation(grid_obj):
    """
    The problem is that it is not possible to merge two Delaunay objects or add additional points.
    Possible strategy is to do a search and find out for which of the tetrahedra of each 3d romboid 
    each point is interior to.
    """
    dict_p_tetrahedron = {}
    n = 3
    gridflat = np.zeros((n**3,) + (3,))
    print(grid_obj.tor_x.shape)
    for ir in range(grid_obj.tor_x.shape[0]-n):
        for it in range(grid_obj.tor_x.shape[1]):
            for ip in range(grid_obj.tor_x.shape[2]-n):
#                 print("iteration # ", ir*it*ip)
                if grid_obj.tor_x.shape[1]-n <= it:
                    nlow = it-grid_obj.tor_x.shape[1]
                    nhigh = nlow + n-1
                    idxs_t = np.linspace(nlow,nhigh,n, dtype = np.int32)
                    gridflat[:,0] = grid_obj.tor_x[ir:(ir + n), idxs_t, ip:(ip + n)].flatten()
                    gridflat[:,1] = grid_obj.tor_y[ir:(ir + n), idxs_t, ip:(ip + n)].flatten()
                    gridflat[:,2] = grid_obj.tor_z[ir:(ir + n), idxs_t, ip:(ip + n)].flatten()
                else:
                    gridflat[:,0] = grid_obj.tor_x[ir:(ir + n), it:(it + n), ip:(ip + n)].flatten()
                    gridflat[:,1] = grid_obj.tor_y[ir:(ir + n), it:(it + n), ip:(ip + n)].flatten()
                    gridflat[:,2] = grid_obj.tor_z[ir:(ir + n), it:(it + n), ip:(ip + n)].flatten()
                tri = Delaunay(gridflat)
                idx_tri = tri.simplices
                coord_tri = gridflat[idx_tri]

                minx = np.min(gridflat[:,0])
                maxx = np.max(gridflat[:,0])
                miny = np.min(gridflat[:,1])
                maxy = np.max(gridflat[:,1])
                minz = np.min(gridflat[:,2])
                maxz = np.max(gridflat[:,2])
#                     print("minx", minx, "maxx", maxx, "miny", miny, "maxy", maxy, "minz", minz, "maxz", maxz)

                epsx = grid_obj.cart_x[1,0,0] - grid_obj.cart_x[0,0,0]
                epsy = grid_obj.cart_y[0,1,0] - grid_obj.cart_y[0,0,0]
                epsz = grid_obj.cart_z[0,0,1] - grid_obj.cart_z[0,0,0]
            
                idx_inside = np.argwhere((minx - epsx < grid_obj.cartflat[:,0]) & (maxx + epsx > grid_obj.cartflat[:,0]) & 
                             (miny - epsy < grid_obj.cartflat[:,1]) & (maxy + epsy > grid_obj.cartflat[:,1]) & 
                             (minz - epsz < grid_obj.cartflat[:,2]) & (maxz + epsz > grid_obj.cartflat[:,2]) )
                for i in idx_inside:
                    p = grid_obj.cartflat[i][0]
                    idx_simplex_embed_p = tri.find_simplex(p)
                    if idx_simplex_embed_p != -1:
                        p_tuple = tuple(list(p))
                        dict_p_tetrahedron[p_tuple] = coord_tri[idx_simplex_embed_p]                

    return dict_p_tetrahedron

In [322]:
tik = time.time()
dict_p_tetrahedron_mantest2 = manual_tessalation(grid_obj = gridlist[1])
len(dict_p_tetrahedron_mantest2)
tok = time.time()
print("The algortihm used", tok-tik, "seconds.")

(20, 20, 40)
The algortihm used 51.05553197860718 seconds.


In [323]:
tik = time.time()
potmesh_mantest2, dict_coord_pot_interpol_mantest2 = make_param_mesh(dict_p_tetra = dict_p_tetrahedron_mantest2, 
                                                     grid_obj = gridlist[1], 
                                                     tpot_torgrid = tpot_torlist[1])
tok = time.time()
print("The algortihm used", tok-tik, "seconds.")

The algortihm used 0.4550361633300781 seconds.


In [325]:
"""
3D plot of the difference between the analytical and the interpolated potential.
"""

fig_testpot_interpol = plt.figure()
ax_testpot_interpol = fig_testpot_interpol.add_subplot(projection='3d')

potscat = ax_testpot_interpol.scatter(gridlist[1].cart_x, 
                                      gridlist[1].cart_y, 
                                      gridlist[1].cart_z, 
                                      c = potmesh_mantest2 - tpot_cartlist[1], cmap = cm.jet)

ax_testpot_interpol.set_xlabel("$x[m]$")
ax_testpot_interpol.set_ylabel("$y[m]$")
ax_testpot_interpol.set_zlabel("$z[m]$")
fig_testpot_interpol.colorbar(potscat)

In [131]:
potmesh_tri_list = []
runtimelist = []
for i in range(4):
    tik = time.time()
    dictpintetra = manual_tessalation(grid_obj = gridlist[i])
    parammesh, dict_param = make_param_mesh(dict_p_tetra = dictpintetra,
                                            grid_obj = gridlist[i],
                                            tpot_torgrid = tpot_torlist[i])
    potmesh_tri_list.append( parammesh )
    tok = time.time()
    runtimelist.append(tok-tik)

(10, 10, 20)
(20, 20, 40)
(30, 30, 60)
(40, 40, 80)


In [135]:
potmesh_tri_list = np.asarray(potmesh_tri_list)
potmesh_tri_list.shape
# type(potmesh_tri_list[1])

(4, 10, 10, 10)

In [144]:
im = plt.imshow(potmesh_tri_list[3,:,5].T)
plt.colorbar(im)

In [52]:
"""
Now want to find a cell using vtk and mayavi.
"""

from tvtk.api import tvtk
from mayavi.scripts import mayavi2
from PyQt5 import QtCore
import sip
import vtk

ok


In [54]:
# # View the data.
# @mayavi2.standalone
# def view():
#     from mayavi.sources.vtk_data_source import VTKDataSource
#     from mayavi.modules.api import Outline, GridPlane

#     mayavi.new_scene()
#     src = VTKDataSource(data=sgrid)
#     mayavi.add_source(src)
#     mayavi.add_module(Outline())
#     g = GridPlane()
#     g.grid_plane.axis = "x"
#     mayavi.add_module(g)
#     g = GridPlane()
#     g.grid_plane.axis = "y"
#     mayavi.add_module(g)
#     g = GridPlane()
#     g.grid_plane.axis = "z"
#     mayavi.add_module(g)


# if __name__ == "__main__":
#     view()

In [326]:
"""
Example vtk.
cellid is the cell find_cell() starts its search
tol2 is the tolerance of the cell found
subid set to default value
pcoord is ?
weights set to default, !!! need to find out how they are computed, but I know it sums to 1
cid is the id of the gridcell from the flattened gridcell array with dims (n_i -1, n_j -1, n_k - 1)
"""

cell = None
# cellid -1 means that the point is not found in the cell
cellid = -1
tol2 = 1e-6
subid = vtk.reference(-1)
pcoord = np.zeros((3))
weights = np.zeros(8)
cid = sgrid.find_cell([2.6,2e-2,2e-2], cell, cellid, tol2, subid, pcoord, weights)
print(cid, subid, pcoord, weights)

855 0 [0.02366018 0.05095581 0.46015286] [0.50021677 0.01212203 0.00065085 0.0268575  0.42637287 0.01033253
 0.00055477 0.02289269]


In [57]:
"""
Find the indices of the cell vertices.
"""
newgridshape = (grid2.tor_x.shape[0]-1, grid2.tor_x.shape[1]-1, grid2.tor_x.shape[2]-1)
ir, it, ip = np.unravel_index(cid, newgridshape)
print(ir, it, ip)


"""
Plot of the vertices of the cell and the interior point [2.6,2e-2,2e-2].
"""
# fig_cell = plt.figure()
# ax_cell = fig_cell.add_subplot(projection='3d')
# ax_cell.scatter(grid2.tor_x[ir:(ir+2), it:(it+2), ip:(ip+2)], 
#                 grid2.tor_y[ir:(ir+2), it:(it+2), ip:(ip+2)], 
#                 grid2.tor_z[ir:(ir+2), it:(it+2), ip:(ip+2)])
# ax_cell.scatter(2.6,2e-2,2e-2)
# ax_cell.set_xlabel("x")
# ax_cell.set_ylabel("y")
# ax_cell.set_zlabel("z")

"""
Here we print the shape of the resulting cell and restructure the data in a mesh desired components.
"""
grid2.tor_x[ir:(ir+2), it:(it+2), ip:(ip+2)].shape
cell_verts_coord = np.zeros((3,2,2,2))
cell_verts_coord[0] = grid2.tor_x[ir:(ir+2), it:(it+2), ip:(ip+2)]
cell_verts_coord[1] = grid2.tor_y[ir:(ir+2), it:(it+2), ip:(ip+2)]
cell_verts_coord[2] = grid2.tor_z[ir:(ir+2), it:(it+2), ip:(ip+2)]
cell_verts_coord

5 0 0


array([[[[2.56      , 2.42129214],
         [2.42898489, 2.29737579]],

        [[2.67      , 2.52533204],
         [2.51324978, 2.37707497]]],


       [[[0.        , 0.83123064],
         [0.        , 0.7886901 ]],

        [[0.        , 0.86694758],
         [0.        , 0.81605087]]],


       [[[0.        , 0.        ],
         [0.35996106, 0.35996106]],

        [[0.        , 0.        ],
         [0.4306677 , 0.4306677 ]]]])

In [58]:
def make_parammesh_vtk(grid_obj, sgrid, tpot_torgrid):
    
    dict_coord_tpot = {}
    for i in range(len(grid_obj.torflat)):
        dict_coord_tpot[tuple(grid_obj.torflat[i])] = tpot_torgrid.flatten()[i]
        
    potmesh = np.nan*np.ones(grid_obj.cart_x.shape)
    
    # pass initial values
    cell = None
    # means not found
    cellid = -1
    tol2 = 1e-6
    subid = vtk.reference(-1)
    pcoord = np.zeros((3))
    weights = np.zeros(8)
    for p in grid_obj.cartflat:
        #find the cell p is interior to
        cid = sgrid.find_cell(p, cell, cellid, tol2, subid, pcoord, weights)
        if cid == -1:
            paramval = np.nan
            continue
        else:
            weights_reshaped = weights.reshape((2,2,2))

            # unravel index of corner point of the cell in the toroidal grid
            newgridshape = (grid_obj.tor_x.shape[0]-1, grid_obj.tor_x.shape[1]-1, grid_obj.tor_x.shape[2]-1)
            ir, it, ip = np.unravel_index(cid, newgridshape)

            # creates a mesh in xyz of the cell vertices
            cell_verts_coord = np.zeros((3,2,2,2))
            cell_verts_coord[0] = grid_obj.tor_x[ir:(ir+2), it:(it+2), ip:(ip+2)]
            cell_verts_coord[1] = grid_obj.tor_y[ir:(ir+2), it:(it+2), ip:(ip+2)]
            cell_verts_coord[2] = grid_obj.tor_z[ir:(ir+2), it:(it+2), ip:(ip+2)]

            #find the potential values of each vertex of the cell and interpolate the data using weights
            vals = []
            for i_r in range(2):
                for i_t in range(2):
                    for i_p in range(2):
                        pval = dict_coord_tpot[tuple([cell_verts_coord[0,i_r, i_t, i_p],
                                                      cell_verts_coord[1,i_r, i_t, i_p],
                                                      cell_verts_coord[2,i_r, i_t, i_p]])]*weights_reshaped[i_r, i_t, i_p]
                        vals.append(pval)
            vals = np.asarray(vals, np.float64)
            paramval = np.sum(vals, axis = 0)
            
            coord_idx = np.argwhere((grid_obj.cart_x == p[0]) & 
                                    (grid_obj.cart_y == p[1]) & 
                                    (grid_obj.cart_z == p[2]))[0] 
            potmesh[coord_idx[0], coord_idx[1], coord_idx[2]] = paramval
    return potmesh

In [146]:
grid_coarse = grid(ngrid_cart = 10, ngrid_tor = 5, R0 = 2)

In [147]:
tpot_tor_coarse = testpot(r = grid_coarse.R)

In [148]:
sgrid_coarse = tvtk.StructuredGrid(dimensions=grid_coarse.tor_x.shape[::-1])
sgrid_coarse.points = grid_coarse.torflat

In [149]:
tik = time.time()
potmesh_vtk_coarse = make_parammesh_vtk(grid_obj = grid_coarse, sgrid = sgrid_coarse, tpot_torgrid = tpot_tor_coarse)
tok = time.time()

In [151]:
im = plt.imshow(potmesh_vtk_coarse[:,5].T)
plt.colorbar(im)

In [152]:
sgrid_list = []
potmesh_vtk_list = []
runtimelist = []
for i in range(4):
    tik = time.time()
    sgrid_list.append(tvtk.StructuredGrid(dimensions=gridlist[i].tor_x.shape[::-1]))
    sgrid_list[i].points = gridlist[i].torflat
    potmesh_vtk_list.append( make_parammesh_vtk(grid_obj = gridlist[i], 
                                                sgrid = sgrid_list[i], 
                                                tpot_torgrid = tpot_torlist[i]) )
    tok = time.time()
    runtimelist.append(tok-tik)

In [156]:
# im = plt.imshow(potmesh_vtk_list[3][:,5].T)
# plt.colorbar(im)

In [158]:
def calc_l2_norm(potmesh, tpot_cart, grid_obj, idir):
    normconst = grid_obj.tor_x.shape[idir]**(3/2)
    return np.sqrt(np.nansum((potmesh - tpot_cart)**2))/normconst

l2s = [[], [], []]
for i in range(4):
    for j in range(3):
        l2s[j].append( calc_l2_norm(potmesh = potmesh_vtk_list[i],
                                    tpot_cart = tpot_cartlist[i],
                                    grid_obj = gridlist[i],
                                    idir = j) )


l2s = np.asarray(l2s)
print(l2s[0])
print(l2s[1])
print(l2s[2])

[1.09841756e-02 8.69265756e-04 2.37531860e-04 7.10618585e-05]
[1.09841756e-02 8.69265756e-04 2.37531860e-04 7.10618585e-05]
[3.88349254e-03 3.07331855e-04 8.39801945e-05 2.51241610e-05]


In [92]:
ngrids_x = np.asarray([x.tor_x.shape[0] for x in gridlist])
ngrids_y = np.asarray([y.tor_y.shape[1] for y in gridlist])
ngrids_z = np.asarray([z.tor_z.shape[2] for z in gridlist])
print(ngrids_x, ngrids_y, ngrids_z)
plt.figure()
plt.plot(1/ngrids_x, l2s[0], '--',label = "$n_{grid, \\hat{r}}$")
plt.plot(1/ngrids_y, l2s[1], '-.',label = "$n_{grid, \\hat{\\theta}}$")
plt.plot(1/ngrids_z, l2s[2], '-',label = "$n_{grid, \\hat{\\phi}}$")
plt.xlabel("$\\frac{1}{n_{grid}}$")
plt.ylabel("$L^2(\\Phi(\\mathbf{r}))$")
plt.xscale("log")
plt.yscale("log")
plt.legend()
plt.show()

[10 20 30 40] [10 20 30 40] [20 40 60 80]


In [315]:
"""
Do exponential fit the gridscaling data compared with the l2 norm.
"""
def expofit(ngx, ngy, ngz, l2list):
    
    ax, bx = np.polyfit(np.log(1/ngx), np.log(l2list[0]), 1, w=np.sqrt(l2list[0]))
    ay, by = np.polyfit(np.log(1/ngy), np.log(l2list[1]), 1, w=np.sqrt(l2list[1]))
    az, bz = np.polyfit(np.log(1/ngz), np.log(l2list[2]), 1, w=np.sqrt(l2list[2]))

    nabcis = ngrids_x#np.linspace(1e1, 1e3, int(1e3))

    plt.figure()
    plt.plot(1/ngrids_x, l2list[0], '--',label = "$n_{grid, \\hat{r}}$")
    plt.plot(1/ngrids_y, l2list[1], '-.',label = "$n_{grid, \\hat{\\theta}}$")
    plt.plot(1/ngrids_z, l2list[2], '-',label = "$n_{grid, \\hat{\\phi}}$")
    plt.plot(1/ngrids_x, np.exp(ax*np.log(1/nabcis) + bx), '--',label = "$\\approx n_{grid, \\hat{r}}$")
    plt.plot(1/ngrids_y, np.exp(ay*np.log(1/nabcis) + by), '-.',label = "$\\approx n_{grid, \\hat{\\theta}}$")
    plt.plot(1/ngrids_z, np.exp(az*np.log(1/nabcis) + bz), '-',label = "$\\approx n_{grid, \\hat{\\phi}}$")
    plt.xlabel("$\\frac{1}{n_{grid}}$")
    plt.ylabel("$L^2(\\Phi(\\mathbf{r}))$")
    plt.xscale("log")
    plt.yscale("log")
    plt.legend()
    plt.show()
    
expofit(ngx = ngrids_x, ngy = ngrids_y, ngz = ngrids_x, l2list = l2s)

In [227]:
"""
Use MMS to confirm that error of grid converges as you increase the number of gridpoints.
Strategy:
1. Establish all grids with increasing number of points in the cartesian grid.
2. Calculate the interpolation
3. Compare potential values with the direct solution on the cartesian grid and compute a 1D measure of the error.
4. Plot error in loglog and see if the error converges.

"""
tik = time.time()

gridlist = []
tpot_cartlist = []
tpot_torlist = []
for i in range(4):
    gridlist.append( grid(ngrid_cart = 20, ngrid_tor = 10*(i+1), R0 = 2) )
    tpot_adir_cart = tpot_alldir_cart(grid_obj = gridlist[i])
    tpot_adir_tor = tpot_alldir_tor(grid_obj = gridlist[i])
    tpot_cartlist.append( tpot_adir_cart )
    tpot_torlist.append( tpot_adir_tor )

sgrid_list = []
potmesh_vtk_list = []
runtimelist = []
for i in range(4):
    tik = time.time()
    sgrid_list.append(tvtk.StructuredGrid(dimensions=gridlist[i].tor_x.shape[::-1]))
    sgrid_list[i].points = gridlist[i].torflat
    potmesh_vtk_list.append( make_parammesh_vtk(grid_obj = gridlist[i], 
                                                sgrid = sgrid_list[i], 
                                                tpot_torgrid = tpot_torlist[i]) )
    tok = time.time()
    runtimelist.append(tok-tik)

In [318]:
runtimelist
plt.figure()
plt.plot([10,20,30,40], runtimelist)

In [230]:
potmesh_vtk_list = np.array(potmesh_vtk_list)
potmesh_vtk_list.shape

(4, 20, 20, 20)

In [237]:
l2s_adir = [[], [], []]
for i in range(4):
    for j in range(3):
        l2s_adir[j].append( calc_l2_norm(potmesh = potmesh_vtk_list[i],
                                         tpot_cart = tpot_cartlist[i],
                                         grid_obj = gridlist[i],
                                         idir = j) )


l2s_adir = np.asarray(l2s_adir)
print(l2s_adir[0])
print(l2s_adir[1])
print(l2s_adir[2])

[0.10682808 0.04224752 0.02398167 0.01580915]
[0.10682808 0.04224752 0.02398167 0.01580915]
[0.03776943 0.01493675 0.0084788  0.00558938]


In [295]:
im = plt.imshow(potmesh_vtk_list[2][:,10].T)
plt.colorbar(im)

In [316]:
expofit(ngx = ngrids_x, ngy = ngrids_y, ngz = ngrids_x, l2list = l2s_adir)